In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam
from keras.layers import (Conv2D, Flatten, MaxPooling2D, Input, 
                          BatchNormalization, Dropout, Dense, InputLayer, GlobalAveragePooling2D)
from keras.models import Model

def create_combined_model(input_shape=(48, 48, 1), num_classes=8):
    vgg16_base = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

    input_layer = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(3, (1, 1), padding='same')(input_layer)  # Repeat grayscale channel to 3 channels
    x = vgg16_base(x)

    # Custom layers added after VGG16
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)  # Removed pooling to prevent dimension reduction

    x = Conv2D(filters=192, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    # Instead of MaxPooling, we use GlobalAveragePooling2D to handle the dimensions properly
    x = GlobalAveragePooling2D()(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    output_layer = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=input_layer, outputs=output_layer, name='combined_model')

2024-06-15 16:36:03.390939: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 16:36:03.391044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 16:36:03.555130: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def train():
    log_dir = './log'
    train_dataset_path = r"/kaggle/input/google-fer-image-format/train"
    val_dataset_path = r'/kaggle/input/google-fer-image-format/val'
    batch_size = 64
    lr = 1e-3
    epochs = 720
    num_classes = 8

    train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=10,
        zoom_range=0.1,
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        directory=train_dataset_path,
        target_size=(48, 48),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    test_datagen = ImageDataGenerator(rescale=1 / 255.0)

    valid_generator = test_datagen.flow_from_directory(
        directory=val_dataset_path,
        target_size=(48, 48),
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    model = create_combined_model(num_classes=num_classes)
    model.summary()

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
    tensorboard = TensorBoard(log_dir=log_dir)
    
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)

    model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=epochs,
        callbacks=[tensorboard, early_stopping]
    )

    model.evaluate(valid_generator, verbose=1)
    model.save('./vgg_custom.h5')

if __name__ == '__main__':
    train()

Found 32645 images belonging to 8 classes.
Found 8166 images belonging to 8 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "combined_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 48, 48, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 1, 1, 256)      │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 1, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 1, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 1, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 1, 192)      │       663,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 1, 192)      │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 1, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 1, 1, 384)      │       663,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1, 1, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1, 1, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 384)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,213,902 (69.48 MB)

 Trainable params: 18,210,958 (69.47 MB)

...

Epoch 1/720


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718469437.067572     129 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1718469437.097886     129 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.1584 - loss: 2.4590

W0000 00:00:1718469557.464018     132 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


511/511 ━━━━━━━━━━━━━━━━━━━━ 179s 289ms/step - accuracy: 0.1584 - loss: 2.4587 - val_accuracy: 0.1286 - val_loss: 3.0486
Epoch 2/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 47s 91ms/step - accuracy: 0.1925 - loss: 2.0688 - val_accuracy: 0.2066 - val_loss: 2.0936
Epoch 3/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 47s 91ms/step - accuracy: 0.2164 - loss: 2.0002 - val_accuracy: 0.2627 - val_loss: 1.8023
Epoch 4/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 46s 90ms/step - accuracy: 0.2570 - loss: 1.8554 - val_accuracy: 0.0699 - val_loss: 31.3341
Epoch 5/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 46s 89ms/step - accuracy: 0.2669 - loss: 1.8126 - val_accuracy: 0.2394 - val_loss: 1.8635
Epoch 6/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 45s 87ms/step - accuracy: 0.2671 - loss: 1.7850 - val_accuracy: 0.2803 - val_loss: 1.7275
Epoch 7/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 47s 90ms/step - accuracy: 0.2629 - loss: 1.7843 - val_accuracy: 0.0643 - val_loss: 51.7985
Epoch 8/720
511/511 ━━━━━━━━━━━━━━━━━━━━ 47s 91ms/step - accuracy: 0.2716 - loss: 1.7687 - 